In [1]:
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import minimize
from scipy import integrate
import numpy as np
import math
import csv
import pandas as pd
from sklearn import linear_model


def mle(func, sample):
    mle_func = lambda params: -sum([math.log(math.fabs(func(x_i, *params))) for x_i in sample])
    
    return minimize(mle_func, [1, 1]).x


def grid_generator(n):
    k = math.ceil(1.72* (n**(1/3)))
    k = 7
    l_b, r_b = -1.1, 1.1
    step = (r_b-l_b)/k
    res = [(-math.inf, l_b)]
    res += [(l_b + step*i, l_b+step*(i+1)) for i in range(k)]
    res += [(r_b, math.inf)]
    return res

def get_n_i(sample):
    
    def n_i(col):
        new_col = []
        c =0 
        
        for cut in col:
            new_col.append(0)
            for i in sample:
                new_col[c] += 0 if (i <= cut[0] or i >= cut[1]) else 1
            c += 1
        return new_col
    
    return n_i


def get_p_i(F):

    def p_i(col):
        new_col = []
        c = 0 
        for cut in col:
            new_col.append(F(cut[1]) - F(cut[0]))
        return new_col
    
    return p_i

        
    
                   
def create_table(sample, F, alpha=0.05):
    ni = get_n_i(sample)
    pi = get_p_i(F)
    frame = pd.DataFrame(columns=['Границы'])
    frame['Границы'] = grid_generator(len(sample))
    frame['n_i'] = ni(frame['Границы'])
    frame['p_i'] = pi(frame['Границы'])
    frame['np_i'] = len(sample) * frame['p_i']
    frame['n_i-np_i'] = frame['n_i'] - frame['np_i']
    frame['(n_i - np_i)^2/(np_i)'] = (frame['n_i'] - frame['np_i'])**2 / (frame['np_i'])
    frame = frame.round(4)
    print(frame)
    print(sum(frame['(n_i - np_i)^2/(np_i)']))
    frame.to_csv('2.csv')
    
    
sample = np.array(stats.norm.rvs(0, 1, size=20))
# sample = stats.laplace.rvs(0, 1, size = 20)

t, y = mle(stats.norm.pdf, sample)
hypotise = lambda x: stats.norm.cdf(x, t, y)

print(t, y)

create_table(sample, hypotise)

# f_test = lambda x: math.fabs(hypotise(x) - stats.norm.cdf(x, 0, 1)) 
# print(integrate.quad(f_test, -200, 200)[0])

# x = np.arange(-3, 3, 0.05)
# y1 = hypotise(x)
# y2 = stats.norm.cdf(x, 0, 1)

# plt.plot(x, y2, label='N(mu, sigma)')
# plt.plot(x, y1, label='N(0, 1)')
# plt.legend()
# plt.show()
# plt.savefig('g.png')


0.022530793040457596 0.7766279605522142
                                       Границы  n_i     p_i    np_i  n_i-np_i  \
0                                 (-inf, -1.1)    2  0.0742  1.4835    0.5165   
1                  (-1.1, -0.7857142857142858)    3  0.0748  1.4966    1.5034   
2   (-0.7857142857142858, -0.4714285714285714)    0  0.1134  2.2675   -2.2675   
3  (-0.4714285714285714, -0.15714285714285703)    0  0.1461  2.9229   -2.9229   
4  (-0.15714285714285703, 0.15714285714285725)    6  0.1603  3.2056    2.7944   
5   (0.15714285714285725, 0.47142857142857153)    4  0.1496  2.9913    1.0087   
6     (0.47142857142857153, 0.785714285714286)    3  0.1187  2.3750    0.6250   
7                     (0.785714285714286, 1.1)    1  0.0802  1.6043   -0.6043   
8                                   (1.1, inf)    1  0.0827  1.6533   -0.6533   

   (n_i - np_i)^2/(np_i)  
0                 0.1798  
1                 1.5102  
2                 2.2675  
3                 2.9229  
4             